In [ ]:
import joblib
import pandas as pd
import numpy as np
import spacy
from textstat import flesch_reading_ease
from textblob import TextBlob

# --- CONFIGURAZIONE MOTORE ---
try:
    nlp = spacy.load("en_core_web_sm", disable=["ner"])
except:
    import os
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm", disable=["ner"])

def blind_test(text_to_test):
    print("🚀 Avvio Analisi Stilometrica Profonda...")
    
    # 1. Caricamento Modelli Salvati
    try:
        model = joblib.load('rf_baseline_model.pkl')
        scaler = joblib.load('feature_scaler.pkl')
        with open('feature_list.txt', 'r') as f:
            features_ordered = [line.strip() for line in f.readlines()]
    except Exception as e:
        print(f"❌ Errore: File mancanti. Assicurati di aver salvato .pkl e feature_list.txt\n{e}")
        return

    # 2. Estrazione Firme (dal tuo notebook signature.ipynb)
    # IMPORTANTE: Usiamo il testo grezzo per non distruggere la sintassi
    signatures = extract_stylometric_signature(text_to_test)
    df_features = pd.DataFrame([signatures])

    # 3. Predizione
    X_scaled = scaler.transform(df_features[features_ordered].values)
    prediction = model.predict(X_scaled)[0]
    probs = model.predict_proba(X_scaled)[0]

    # 4. Report Grafico dei Risultati
    label = "🤖 INTELLIGENZA ARTIFICIALE" if prediction == 1 else "✍️ ESSERE UMANO"
    confidenza = probs[1] if prediction == 1 else probs[0]

    print("\n" + "—"*45)
    print(f"RISULTATO FINALE: {label}")
    print(f"CONFIDENZA: {confidenza:.2%}")
    print("—"*45)
    
    print(f"\nDETTAGLIO DNA STILOMETRICO:")
    print(f"• Profondità Sintattica: {signatures['dependency_depth_mean']:.3f} (AI tipicamente > 2.5)")
    print(f"• Entropia POS: {signatures['pos_bigram_entropy']:.3f} (Umano tipicamente più alta/caotica)")
    print(f"• Template Bias: {signatures['template_bias_score']:.2f} (IA usa più connettivi logici)")
    print("—"*45)

# --- ESEMPI DI VERIFICA ---
print("🧪 TEST 1: Testo generato da AI")
testo_ai = "The implementation of artificial intelligence in modern workflows significantly enhances productivity. Furthermore, it is important to note that efficiency remains a primary goal."
blind_test(testo_ai)

print("\n🧪 TEST 2: Testo scritto da Umano")
testo_umano = "I don't know... maybe it's just me, but I feel like this whole thing is a bit too much. It's weird, right? Let me know what you think about it tomorrow."
blind_test(testo_umano)

testo_formale = """The High Contracting Parties, recognizing that the community of nations 
is governed by the principles of international law, including the principle that 
states shall refrain in their international relations from the threat or use of force 
against the territorial integrity or political independence of any state, have 
agreed as follows: The present Convention shall be open for signature by all 
States Members of the United Nations."""

blind_test(testo_formale)

testo_mimetico = """so... i was thinking about that trip we planned. idk, maybe 
we should just stay home? its not like i dont want to go, but work has been 
crazy and i'm just... exhausted. anyway, let me know what u think. 
maybe we can grab a coffee later and talk about it properly??"""

blind_test(testo_mimetico)
